In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} pygraphviz

# Ülesande püstitus

Proovime lihtsamat ülesannet. Vaata kuidas verb + määrsõna teiste nimisõnadega kombineerub
    
     kirjutas lepingule alla 

Ka siin tekib sul palju jama, sest sul võivad olla aja ja koha määrused sees, aga  proovi eraldada andmeid kujul

kirjutas alla -->{lepingule:obl, jaanuaris:obl}
kirjutas alla -->{dokumedile:obl}

st jäta alles kõik lapstipud mis pole alus ja märgi ära ka vastav deprel annotatsioon

Äkki õnnestub selle peale vaadates  osa ebaolulistest määrustest välja lõigata ja see subcat seos sealt välja õngitseda  


## Täpsustus

Võtame sisse ainult compound:prt verbi alluvad, kui seda pole, siis läheb verb "üksi kirja"

Võtame sisse nimisõnad, asesõnad, pärisnimed ja nende käänded

Lisame info, kas verb oli eitusega (feat sisaldab neg väärtust). Kui on eitav, siis muudame verbi kujule "ei VERB"

"alus" ei jää välja.

## Vajalikud teegid

In [ ]:
from __future__ import print_function

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict
import matplotlib.pyplot as plt
import random

#import etnltk
#from estnltk.storage.postgres import PostgresStorage

import networkx as nx
import sqlite3
import sys
import json

## Abifunktsioonid

In [ ]:
class graphFunctions:

    # kahe listi ühisosa 
    def intersection(a, b):
        return list(set(a).intersection(b))

    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    def drawGraph(G):
        # joonise suurus, et enamik puudest ära mahuks
        plt.rcParams["figure.figsize"] = (18.5, 10.5)
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        plt.show()


In [ ]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

In [ ]:
def getCase(arr):
    for attr in arr:
        if attr in ('nom' #nimetav
                    , 'gen' #omastav
                    , 'part' #osastav
                    , 'adit' #lyh sisse
                    , 'ill' #sisse
                    , 'in'#sees
                    , 'el' #seest
                    , 'ad' #alale
                    , 'all' #alal
                    , 'abl' #alalt
                    , 'tr' #saav
                    , 'term' #rajav
                    , 'es' #olev
                    , 'abes'  #ilma#
                    , 'kom' #kaasa#
                   ) :
            return attr

def getNumber(arr):
    for attr in arr:
        if attr in ('sg' #ainsus
                    , 'pl' #mitmus
                   ):
            return attr


### Baasi  (sqlite3)  ettevalmistamine täitmiseks

Luuakse tabelid:
* collections_processed - salvestatakse viimane salvestatud collection Id
* {TABLENAME} - tabel kollokatsioonidega
 * lemma1 text
 * pos1 text
 * lemma2 text
 * pos2 text
 * deprel text
 * children json
 * total integer
 * example1 text
 * example2 text
 * example3 text

Luuakse indeksid:
* collections_processed_uniq
* {TABLENAME}_col1_col2_unique

In [ ]:
def prepCollDb(TABLENAME, cursor, conn):
  

    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq ON collections_processed(tablename);
    """)
    
    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, 0, 0,) )
    
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {TABLENAME}
                      (lemma1 text, pos1 text, lemma2 text, pos2 text, deprel text, children json, total integer, example1 text, example2 text, example3 text);
                      """)


    cursor.execute(f"""DELETE FROM {TABLENAME};""")
    
    INDEXNAME = f'col1_col2_unique'
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}
        ON {TABLENAME}(lemma1, pos1, lemma2, pos2, deprel, children);
        """)

   
    conn.commit()



Tulemuse salvestamine baasi (sqlite3)

In [ ]:
def saveCollToDb(TABLENAME, cursor, conn, collocations, examples, lastcollection):
   
    sqlColls = []
   
    
    for key in collocations.keys():
        #print ('key', key)
        #save key in parents or update count, get rowid
        #lemma1 text, pos1 text, lemma2 text, pos2 text, total integer, deprel
     
        example1 = None
        example2 = None
        example3 = None

        if len(examples[key])>0:
            example1 = examples[key][0]
        if len(examples[key])>1:
            example2 = examples[key][1]
        if len(examples[key])>2:
            example3 = examples[key][2]
        
        lemma1 = key[0]
        pos1 = key[2]
        lemma2 = key[1]
        pos2 = key[3]
        deprel = key[4]
        children = key[5]
        
        sqlColls.append( (lemma1, pos1, lemma2,  pos2, deprel, children, collocations[key], example1, example2, example3 , collocations[key], example1, example2,example3,) )

    cursor.executemany(f"""
        INSERT INTO {TABLENAME} VALUES (?,?,?,?,?,?,?,?,?,?)
            ON CONFLICT(lemma1, pos1, lemma2, pos2, deprel, children)
            DO UPDATE SET total=total+? 
            ,  example1= ?
            ,  example2= ?
            ,  example3= ?
            ;""", sqlColls)

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection,) )
    
    conn.commit()
    eprint(f'andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - {lastcollection}' )
    

###  sõltuvuste leidmine 

* Kõik kolmikud, kus 
    * ülemus on tipp1 sõnaliigiga V ja alluv on tipp2 deprel compound:prt
    * ülemusel tipp1 on nimisõna/asesõna/pärisnimi (S/P/H) alluv, selle deprel ja kääne 

#### Näide 1


saama	V	välja	D	compound:prt

In [ ]:
def extract_verb_adverb_custom_deprel(G, collocations, examples, sentence):
   
    
    #graphFunctions.drawGraph(G)
    #print (sentence)
    # lyhim tee tippude vahel
    path = nx.all_pairs_shortest_path_length(G)
    path_reversed = nx.all_pairs_shortest_path_length(G.reverse())

    # kauguste maatriksid
    dpath = {x[0]:x[1] for x in path}
    dpath_reversed = {x[0]:x[1] for x in path}

    #eprint('eraldame tipud vajalike parameetritega ')
    verbnodes = graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'V')
    
    #compound tipid
    compoundnodes = graphFunctions.get_nodes_by_attributes(G, attrname = 'deprel', attrvalue = 'compound:prt')
    
    nouns = graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'S') + graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'H')+ graphFunctions.get_nodes_by_attributes(G, attrname = 'pos', attrvalue = 'P')
    
    cases = graphFunctions.get_nodes_by_attributes(G, attrname = 'deprel', attrvalue = 'case')
    
    #print ('cases', cases)
    #liigume tegusõnade kaupa
    
    #see näide on vananenud, kogume ainult compound:prt tippe
    #tekitame kollokatsioonipaari sellise:
    ##Isa hülgas Jimmy kohe pärast tema sündi .
    ## hülgama -[ advmod ]-> kohe sellel on id ja count?
    ## children: obj Jimmy H nom
    ## children: obj sünd S com pärast D
    #    #-[ obj ]-> Jimmy nom

     #   #-[ obl ]-> sündi com -[ case ]-> pärast
    

    for verb in verbnodes:
        #print (G.nodes[verb]['lemma'])
        childcompounds = []
        childnouns = []
        for compound in compoundnodes:
            #kui on vahetu alluv
            if compound in dpath[verb] and dpath[verb][compound]==1: childcompounds.append(compound)
        
        
        # kui ei ole compounds alluvaid, siis l'heb verb üksi kirja
        # if not len(childcompounds):
        #     #print ('pole compounds')
        #    key = ( 'ei ' + G.nodes[verb]['lemma'] if 'neg' in G.nodes[verb]['feat'] else G.nodes[verb]['lemma'], '', G.nodes[verb]['pos'], '', '', json.dumps({}) ,)
  
            
        for noun in nouns:
            #kui on vahetu alluv        
            if noun in dpath[verb] and dpath[verb][noun]==1: childnouns.append(noun)

        #kui ei ole compound adverbe, siis lisame ühe rea, kus kõik nimisõnad sees käänetega
        if not len(childcompounds):
            childcompounds.append('-')
            
        for compound in childcompounds:

            #print ('collocation', key)
            children = []
            for noun in childnouns:
                child = {
                        'lemma': G.nodes[noun]['lemma']
                        , 'pos': G.nodes[noun]['pos']
                        , 'case':getCase(G.nodes[noun]['feat'])
                        , 'number':getNumber(G.nodes[noun]['feat'])
                        , 'deprel': G.nodes[noun]['deprel']
                        , 'case_children':[]
                        }
                #print ("\t" '-[',G.nodes[noun]['deprel'],']->', G.nodes[noun]['text'], getCase(G.nodes[noun]['feat']))
                #print ()
                childcases = []
                for case in cases:
                    if case in dpath[noun] and dpath[noun][case]==1: 
                        child['case_children'].append( (G.nodes[case]['lemma'], G.nodes[case]['pos'], ) )
                children.append(child)
            

            key = ( 
                    'ei ' + G.nodes[verb]['lemma'] if 'neg' in G.nodes[verb]['feat']  else G.nodes[verb]['lemma']
                    , '' if compound == '-' else G.nodes[compound]['lemma']
                    , G.nodes[verb]['pos']
                    , '' if compound == '-' else G.nodes[compound]['pos']
                    , '' if compound == '-' else G.nodes[compound]['deprel']
                    , json.dumps(children) ,)

 
            #print (key) 
            if not key in collocations:
                collocations[key] = 0
            if not key in examples:
                examples[key] = []
            collocations[key] += 1
            examples[key].append(sentence)
            if len(examples[key]) > 3:
                del(examples[key][random.randint(0, 2)])
        
                
            
    
    return (collocations, examples)



## Muutujad

**Korpuse TSV fail**

Lähte TSV-faili ja DB tabeli nimi, kuhu tulemus salvestatakse (TSV genereerimise kood on failis **collect_texts_db_tsv.ipynb**)

In [ ]:
#collectionName =  'koondkorpus_base_subset_of_5000_v2' 
collectionName =  'koondkorpus_base_subset_of_5000_v2_short'
#collectionName =  'koondkorpus_base_subset_of_5000_v2' 
corporafile = f'{collectionName}.tsv'
#corporafile = f'kadrile_test.tsv'
#corporafile = '/Volumes/Selena/Kollokatsioonid/koondkorpus/koondkorpus_base_v2_20220216.tsv'


**Kollokatsiooni tüüp**

In [ ]:
TYPE = 'kadrile_verb_adverb_noun_deprel'

**Tabeli nimi andmebaasis**

In [ ]:
TABLENAME = f'all_in_one'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus andmebaasi.

In [ ]:
BATCHSIZE = 1000

## Andmebaasi loomine ja ette valmistamine

In [ ]:
conn = sqlite3.connect(f"{TYPE}_collocations.db") #
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
prepCollDb(TABLENAME, cursor, conn)

## Kollokatsioonide leidmine

Andmed loetakse TSV failist. Igast lausest tehakse *networkx* suunatud graaf. Graafi servadeks on süntaksipuu 
head->child seos. Tippudeks on lause sõnad. 
Tipu omadused on:
* *id* - sõna id
* deprel
* lemma
* pos


In [ ]:

unsaved = 0
collocations = {}
examples = {}
word_id = 0
count = 0
current_sentence = []

#
G = None
colId = None
prevCol = 0
f = TSV.split("\n")

with open(corporafile) as f:

    for line in f:
        count +=1
        line = line.strip("\n")
        row = line.split('\t')
        if not len(row) == 9:
            continue
        data = {}
        prevCol = colId
        (colId, data['start'], data['id'], data['text'], data['lemma'], data['upostag'], data['deprel'], data['head'], data['feat']) = row
        colId = int(colId)
        for k in ('id', 'start', 'head'):
            data[k] = int(data[k])

        word_id +=1
        #sentence start
        if data['id'] == 1:
            if not G==None:
                current_sentence_text = ' '.join([s['text'] for s in current_sentence])
                (collocations, examples) = extract_verb_adverb_custom_deprel(G, collocations, examples, current_sentence_text)
                unsaved = 1
            current_sentence = []
            #count+=1
            word_id +=1
            G = nx.DiGraph()
            if not prevCol ==colId and not colId==0 and not colId%BATCHSIZE:
                saveCollToDb(TABLENAME, cursor, conn, collocations, examples, colId)
                collocations = {}
                unsaved = 0


        #paneme graafi kokku
        G.add_node(word_id, id=data['id'], text=data['text'],  lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'], feat=data['feat'].split('||'))
        G.add_edge(word_id - data['id'] + data['head'], word_id, deprel = data['deprel'])
        current_sentence.append(data)
        
        
    
    #viimane lause
    current_sentence_text = ' '.join([s['text'] for s in current_sentence])
    (collocations, examples) = extract_verb_adverb_custom_deprel(G, collocations, examples, current_sentence_text)
    saveCollToDb(TABLENAME, cursor, conn, collocations, examples, colId)


### Andmebaasi indeksite lisamine


Tehakse kõige viimasena, et andmete sisestamine andmebaasi oleks kiirem.

In [ ]:

indexesQ = [
    f'CREATE INDEX IF NOT EXISTS "deprel" ON {TABLENAME} ("deprel");'
    , f'CREATE INDEX IF NOT EXISTS "lemma1" ON {TABLENAME} ("lemma1");'
    , f'CREATE INDEX IF NOT EXISTS "lemma2" ON {TABLENAME}("lemma2");'
    , f'CREATE INDEX IF NOT EXISTS "pos1" ON {TABLENAME} ("pos1");'
    , f'CREATE INDEX IF NOT EXISTS "pos2" ON {TABLENAME} ("pos2");'
    , f'CREATE INDEX IF NOT EXISTS "total" ON {TABLENAME} ("total" DESC);']

for q in indexesQ: cursor.execute(q)

In [ ]:
cursor.execute(f"SELECT count(*) FROM {TABLENAME}")
all_collocations = cursor.fetchall()

In [ ]:
all_collocations

# Info koondamine

Avame uue andmebaasi.
Loome tabeli koondatud.
Loeme vanast tabelist batchide kaupa.
Eemaldame ebavajaliku info, salvestame koondatud read uude tabelisse


In [ ]:
conn2 = sqlite3.connect(f"{TYPE}_generalized.db") 
cursor2 = conn2.cursor()
TABLE = 'all_generalized'
prepCollDb(TABLE, cursor2, conn2)


In [ ]:
cursor.execute(f"SELECT count(*) AS total FROM all_in_one")
res = cursor.fetchall()
TOTALROWS = res[0]['total']

BATCHSIZE = 100000
DONE = 0


BATCHSIZE = 1000
DONE = 0


In [ ]:
import json
collocations2 = {}
examples2={}
offset = 1

print (f'TOTAL ROWS: {TOTALROWS}')
while offset <= TOTALROWS:
    query = f"SELECT *   FROM all_in_one   LIMIT {BATCHSIZE} OFFSET {offset} "
    cursor.execute(query)
    print (query)
    count = 0
    for r in cursor.fetchall():
        count+=1
        new_children = []
        children = r['children']
        try:
            children= json.loads(children)
        except:
            eprint (f'JSON parsing error on line {count}')
            children = []


        for ch in children:
            #print (ch)
            gen_child = []
            for p in ['case','number','deprel']:
                gen_child.append(ch[p])
            #print(gen_child)
            new_children.append(gen_child)


        key = ( r['lemma1']
                , r['lemma2']
                , r['pos1']
                , r['pos2']
                , r['deprel']
                , json.dumps(new_children) ,)
        if key in collocations2:
            collocations2[key] += r['total']
        else:
            collocations2[key] = r['total']
            examples2[key] = []
        if r['example1']:
            examples2[key].append(r['example1'])
        if r['example2']:
            examples2[key].append(r['example2'])
        if r['example3']:
            examples2[key].append(r['example3'])
        examples2[key] = list(set(examples2[key]))
      

    
    saveCollToDb(TABLE, cursor2, conn2, collocations2, examples2, offset + BATCHSIZE)
    collocations = {}
    unsaved = 0
    
    offset = offset + BATCHSIZE
    
    




In [ ]:
conn2.close()

# Lihtverbid koondatud

* Koondatud info baasist heidame välja morf info, jätame ainult deprel 
* Andmed loeme TSV-failist ja salvestame TSV faili
* Kustutame deprel 'parataxis' ja 'conj' - need ei sõltu verbist
* Kui deprel on tühjad, siis kirjutame 'NULL'

In [1]:
import pandas as pd
import sqlite3
import json
import csv

In [2]:
DBDataFilename = '/Volumes/Selena/Kollokatsioonid/kadrile_verb_adverb_compound_children/kadrile_verb_adverb_noun_deprel_generalized_20221014.db'

In [3]:
from urllib.request import pathname2url

try:
    dburi = 'file:{}?mode=rw'.format(pathname2url(DBDataFilename))
    conn3 = sqlite3.connect(dburi, uri=True)
    conn3.row_factory = sqlite3.Row
    cursor3 = conn3.cursor()
    print ('Connected')
except sqlite3.OperationalError:
    # handle missing database case
    print (f'DB file not found')
    



Connected


In [4]:
cursor3.execute(f"SELECT count(*) AS total FROM all_generalized  WHERE lemma2 = ''")
res = cursor3.fetchall()

BATCHSIZE = 500000
TOTALROWS = res[0]['total']

TOTALROWS

3647171

In [11]:
collocations3 = {}
examples3={}
offset = 1

print (f'TOTAL ROWS: {TOTALROWS}')
while offset <= TOTALROWS:
    query = f"SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT {BATCHSIZE} OFFSET {offset} "
    cursor3.execute(query)
    print (query)
    count = 0
    for r in cursor3.fetchall():
        count+=1
        new_children = []
        children = r['children']
        try:
            children= json.loads(children)
        except:
            eprint (f'JSON parsing error on line {count}')
            children = []

        for ch in children:
            new_children.append(ch[2])
        if 'parataxis' in new_children:
            new_children.remove('parataxis')
        if 'conj' in new_children:
            new_children.remove('conj')
        if not len(new_children): new_children.append('NULL')

        key = ( r['lemma1']
                , r['pos1']
                , ','.join(sorted(new_children)) ,)

        if key in collocations3:
            collocations3[key] += r['total']
        else:
            collocations3[key] = r['total']
            examples3[key] = []
        #if r['example1']:
        #    examples3[key].append(r['example1'])
        #if r['example2']:
        #    examples3[key].append(r['example2'])
        #if r['example3']:
        #    examples3[key].append(r['example3'])
        #examples3[key] = list(set(examples3[key]))
      
    offset = offset + BATCHSIZE
    
   

TOTAL ROWS: 3647171
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 1 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 500001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 1000001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 1500001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 2000001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 2500001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 3000001 
SELECT *   FROM all_generalized WHERE lemma2 = '' LIMIT 500000 OFFSET 3500001 


In [12]:
resultfilename = 'kadrile_lihtverb_deprel_sorted.tsv'
import csv

with open(resultfilename, 'w', newline='') as f:
    csvwriter = csv.writer(f, delimiter='\t')
    for k, v in  sorted(collocations3.items(), key = lambda x: x[1], reverse = True) : 
        row = list(k)
        row.append(v)
        
        
        csvwriter.writerow(row)
f.close()